# 환경설정
- 구글 코랩

In [1]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [2]:
strategy = tf.distribute.TPUStrategy(resolver)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Hanium_private/model/

/content/drive/MyDrive/Hanium_private/model


# KoGPT2

In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.7 MB/s eta 0:00:00


In [16]:
# books_to_embedding.py
import torch
from transformers import PreTrainedTokenizerFast, GPT2Model
import json
import sqlite3

## 모델 및 토크나이저 초기화

In [7]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [8]:
# 모델 및 토크나이저 초기화
MODEL_NAME = 'skt/kogpt2-base-v2'
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME,
                                          bos_token=BOS, eos_token=EOS, unk_token='<unk>',
                                          pad_token=PAD, mask_token=MASK)
model = GPT2Model.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [9]:
model.eval()

GPT2Model(
  (wte): Embedding(51200, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

## embedding vector

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
def create_embedding(text):
    input_ids = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding='max_length').to(device)

    with torch.no_grad():
        output = model(**input_ids)
    last_hidden_state=output.last_hidden_state

    # mean pooling
    attention_mask = input_ids.attention_mask
    num_of_tokens = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

    sum_embeddings = torch.sum(last_hidden_state * num_of_tokens, 1)

    total_num_of_tokens = num_of_tokens.sum(1)
    total_num_of_tokens = torch.clamp(total_num_of_tokens, min=1e-9)

    result = sum_embeddings / total_num_of_tokens
    return result.squeeze()

# 결과값 torch .pt로 저장

### 2. 최종 적용한 방법
- db에서 읽는 방법 (Json > sqlite3) json.load

- JSON

In [12]:
# JSON
# def store_embeddings_to_db(folder):
#     # 모든 도서 원문의 임베딩 계산
#     all_embeddings = []

#     # 여러 JSON 파일을 순회하며 처리
#     for dir in folder:
#         json_directory = dir
#         for filename in sorted(os.listdir(json_directory)):
#             if filename.endswith(".json"):
#                 filepath = os.path.join(json_directory, filename)

#                 # JSON 파일 읽기
#                 with open(filepath, 'r', encoding='utf-8') as file:
#                     data = json.load(file)
#                     passage_id = data["passage_id"]
#                     embedding = create_embedding(data["passage"])
#                     all_embeddings.append(embedding)

#     # 모든 임베딩을 하나의 텐서로 저장
#     all_embeddings_tensor = torch.stack(all_embeddings)
#     torch.save(all_embeddings_tensor, 'embeddings.pt')

In [18]:
def store_embeddings_to_db(folder):
    # 모든 도서 원문의 임베딩 계산
    all_embeddings = []
    passage_ids = []

    # SQLite3 데이터베이스 연결
    conn = sqlite3.connect('data/books.db')
    cursor = conn.cursor()

    # sample 테이블에서 passage_id와 passage 정보 가져오기
    cursor.execute("SELECT id, passage FROM sample") # 'id', 'title_id', 'title', 'passage', 'author', 'type', 'summary'
    data = cursor.fetchall()

    for passage_id, passage in data:
        all_embeddings.append(create_embedding(passage))
        passage_ids.append(passage_id)

    # Embeddings와 passage_ids를 저장
    torch.save(all_embeddings, 'data/embeddings.pt')
    torch.save(passage_ids, 'data/passage_ids.pt')

    return passage_ids, all_embeddings

In [19]:
folder = ['2020-02-019_document_summary_Sample/'] # ['Training/labelingData/', 'Training/originalData/', 'Validation/labelingData/', 'Validation/originalData/']

In [20]:
temp_id, temp_emb = store_embeddings_to_db(folder)

## 결과 분석

In [21]:
temp_id

['CNTS-00047965957_40_41-0',
 'CNTS-00047966059_168',
 'CNTS-00047966059_371',
 'CNTS-00047966059_451',
 'CNTS-00047966059_497',
 'CNTS-00047966059_589',
 'CNTS-00047966059_604',
 'CNTS-00047966059_7',
 'CNTS-00047966265_187_187-0',
 'CNTS-00047966265_240_241-0',
 'CNTS-00047966265_285_285-0',
 'CNTS-00047966265_466_467-0',
 'CNTS-00047966265_481_481-1',
 'CNTS-00047966265_483_483-0',
 'CNTS-00047966265_496_496-0',
 'CNTS-00047966265_499_499-1',
 'CNTS-00047966265_95_95-0',
 'CNTS-00047966745_129_130-0',
 'CNTS-00047966750_168',
 'CNTS-00047966750_180',
 'CNTS-00047966750_262',
 'CNTS-00047966750_342',
 'CNTS-00047966750_366',
 'CNTS-00047966750_489',
 'CNTS-00047966808_137',
 'CNTS-00047966809_6',
 'CNTS-00047966809_79',
 'CNTS-00047966896_254',
 'CNTS-00047966914_191',
 'CNTS-00047966914_215',
 'CNTS-00047966914_372',
 'CNTS-00047966914_39',
 'CNTS-00047966920_134_135-0',
 'CNTS-00047966920_297_298-0',
 'CNTS-00047966920_350_351-0',
 'CNTS-00047966920_401_402-0',
 'CNTS-00047967301_1

In [26]:
len(temp_id)

2000

In [22]:
temp_emb

Output hidden; open in https://colab.research.google.com to view.

In [25]:
len(temp_emb)

2000